# Retrouver dans  Zenodo les jeux de données de son établissement 

In [1]:
import requests, json

In [2]:
# a gist example : https://gist.github.com/slint/d30fc0f415300876facbbeb8a0989ab2

l'API de Zenodo nécessite une clé. cf. https://developers.zenodo.org/#quickstart-upload

In [3]:
with open("../hide/personnal.json") as f : 
    ACCESS_TOKEN = json.load(f)["ZENODO_KEY"]

## Défnir la recherche sur le champs affiliation

cf. https://help.zenodo.org/guides/search/

In [4]:
aff_query = "(univ* AND (diderot OR descartes))"

In [5]:
r = requests.get("https://zenodo.org/api/records",
                 params = {"q" : "creators.affiliation:" +  aff_query,
                           "type" : "dataset",
                           "size" : 300,
                           "sort" : "mostrecent",
                           "all_version" : False,
                           "access_tpoken" : ACCESS_TOKEN
                          })
res = r.json()

Se familiariser avec la structure de la réponse

In [ ]:
# export en JSON
with open("zenodo_raw.json", 'w') as f : 
    json.dump(res, f)

In [ ]:
# impression json
print(json.dumps(res["hits"],  indent = 1))

In [6]:
# nb de jeux de données trouvés
print(res["hits"]["total"])

70


In [ ]:
# imprimer le contenu d'un seul hits
print(json.dumps(res["hits"]["hits"][0], indent = 2))

In [7]:
# imprimer les clés d'un hits
first_hit = res["hits"]["hits"][0]
first_hit.keys()

dict_keys(['conceptdoi', 'conceptrecid', 'created', 'doi', 'files', 'id', 'links', 'metadata', 'owners', 'revision', 'stats', 'updated'])

## Déduire les informations à récupérer
``` 
id, l'identifiant zenodo, pour déduire le lien https://zenodo.org/record/5005868
metadata/publication_date : date du dépot
metadata/titre : titre du jeu de de donnée
metadata/license : la licence apposée
metadata/keywords
metadata/doi
```

## Récupérer les données

In [8]:
# un dict pour coller les données
data = {}

for idx, item in enumerate( res["hits"]["hits"] ) : 
    
    data[idx] = {}
    data[idx]["id"] = item["id"]
    
    data[idx]["title"] = item["metadata"].get("title")
    data[idx]["publication_date"] = item["metadata"].get("publication_date")
    data[idx]["doi"] = item["metadata"].get("doi")
    
    #licence
    if item["metadata"].get("license") : 
        data[idx]["license"] = item["metadata"]["license"]["id"]   
    
    # mots-clés
    if item["metadata"].get("keywords") : 
        data[idx]["keywords"] = ",".join(item["metadata"]["keywords"])  

In [9]:
print( json.dumps(data[0], indent = 1))

{
 "id": 5005868,
 "title": "A global inventory of solar photovoltaic generating units - dataset",
 "publication_date": "2021-10-27",
 "doi": "10.5281/zenodo.5005868",
 "license": "CC-BY-4.0",
 "keywords": "photovoltaic solar,remote sensing,geospatial data,computer vision"
}


## Transform json to df and vizualise

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(data)

In [ ]:
df.sample(2)

In [12]:
# renverser colonne & ligne
df = df.T
# trier par date de dépôt
df = df.sort_values( by = "publication_date", ascending = False)

In [13]:
df.sample(10)

,id,title,publication_date,doi,license,keywords
64,14595,MD simulation trajectory and related files for...,2015-01-27,10.5281/zenodo.14595,GPL-3.0,NaN
22,4946518,Data from: Developmental venous anomaly in dif...,2019-09-06,10.5061/dryad.ps00rd5,CC0-1.0,"MRI,Primary brain tumor,Other cerebrovascular ..."
56,46479,Benchmark of the SAXS Merge method,2013-12-14,10.5281/zenodo.46479,LGPL-2.1,"Integrative Modeling Platform (IMP),Benchmark,..."
24,4900655,Mobiliscope,2021-05-28,10.5281/zenodo.4900655,CC-BY-4.0,social inequalities; segregation; daily mobili...
26,4540395,"Dataset related to article ""Cutaneous barrier ...",2021-02-14,10.5281/zenodo.4540395,NaN,"LPS,RAG,T cells,chemokines,cytokines,dysbiosis..."
20,4954332,Data from: The loci of repeated evolution: a c...,2013-02-04,10.5061/dryad.v66p0,CC0-1.0,"Mutations,Eukaryota,Evolutionary genomics"
4,5089991,Data from: Evolution of a dominant natural iso...,2019-02-02,10.5061/dryad.v5374,CC0-1.0,"human gut,replication rate,plasmid,mutation rate"
1,5547311,Energetic return on investment determines over...,2021-10-04,10.5281/zenodo.5547311,CC-BY-4.0,"soil,microorganisms,microbial activity,organic..."
27,4319024,Example Datasets for Iliski - Neuronal calcium...,2019-09-11,10.5281/zenodo.4319024,CC-BY-4.0,"neurovascular coupling,functional ultrasound,t..."
8,5007851,Data from: Should the WHO growth charts be use...,2016-02-19,10.5061/dryad.8s96r,CC0-1.0,"French references,Growth monitoring,children,W..."


In [ ]:
df.to_csv("data_from_zenodo.csv", index = False)

----
## Vigilance

* la recherche dans l'affiliation se fait pour tous les auteurs (comme le wos) donc du bruit peut être présent en sortie

* l'API donne les résultats avec un systeme de pagination. Vérifier à la fin de la réponse si `links` ne contient pas une url `next`